In [20]:
import json
import os
from collections import Counter

import numpy as np

In [21]:
from invargen.data.expression import *
from invargen.data.tokens import *
from invargen.data.tree import *

In [22]:
from invargen.data.expression import *
from invargen_generic.features import *

In [23]:
from gplearn.fitness import make_fitness
from gplearn.functions import make_function
from gplearn.genetic import SymbolicRegressor

In [24]:
from collections import namedtuple
GenericOperator = namedtuple('GenericOperator', ['name', 'function', 'arity'])

In [25]:
generic_funcs = []

from invargen_generic.operators import funcs as generic_funcs

funcs = [make_function(**func._asdict()) for func in generic_funcs]    

def const(cls):
    
    def _calc(a):
        
        n = len(a)
    
        return np.array([f'Mul(Constant({str(cls)}),{a[i]})' for i in range(n)])

    return _calc
        
const_ops = [i for i in range(4, 5)]
    
#funcs = {}
# Annotate unprotected ops
for op in const_ops:
    
    funcs += [make_function(**GenericOperator(function=const(str(op)), name=str(op), arity=1)._asdict())]
    
        
generic_funcs, funcs

([GenericOperator(name='Add', function=<function binary.<locals>._calc at 0x0000023789B290D0>, arity=2),
  GenericOperator(name='Sub', function=<function binary.<locals>._calc at 0x0000023789B29160>, arity=2),
  GenericOperator(name='Mul', function=<function binary.<locals>._calc at 0x0000023789B291F0>, arity=2)],
  <gplearn.functions._Function at 0x237fdb5ed60>])

In [26]:
seed = 4

In [27]:
import torch
from torch import Tensor

device = torch.device('cpu')

In [28]:
from invargen_qlib.poly_data import PolyData

data = PolyData(device=device)

data_1 = PolyData(device=device)

data_2 = PolyData(device=device)

data_3 = PolyData(device=device)

data_4 = PolyData(device=device)

In [29]:
data.data

tensor([[[  1.,   1.,   1.,  ...,  49.,  51.,  51.],
         [  0.,   1.,   0.,  ...,   1., -35.,  35.],
         [  1.,   2.,   2.,  ...,  51.,  55.,  55.]]])

In [30]:
# first transform under (0, 1//-1, 0)

data_1.data[0][[0, 2]] = data_1.data[0][[2, 0]]

data_1.data[0][[1]] = -data_1.data[0][[1]]

data_1.data

tensor([[[  1.,   2.,   2.,  ...,  51.,  55.,  55.],
         [ -0.,  -1.,  -0.,  ...,  -1.,  35., -35.],
         [  1.,   1.,   1.,  ...,  49.,  51.,  51.]]])

In [31]:
# second transform under (1, 1//0, 1)

data_2.data[0][[2]] = data_2.data[0][[2]] + 1 * data_2.data[0][[1]] + data_2.data[0][[0]]

data_2.data[0][[1]] = data_2.data[0][[1]] + data_2.data[0][[0]] * 2

data_2.data

tensor([[[  1.,   1.,   1.,  ...,  49.,  51.,  51.],
         [  2.,   3.,   2.,  ...,  99.,  67., 137.],
         [  2.,   4.,   3.,  ..., 101.,  71., 141.]]])

In [32]:
# second transform under (1, 1//0, 1)

data_3.data = data.data * 2

data_3.data

# second transform under (1, 1//0, 1)

data_4.data = data.data * 4

data_4.data

tensor([[[   4.,    4.,    4.,  ...,  196.,  204.,  204.],
         [   0.,    4.,    0.,  ...,    4., -140.,  140.],
         [   4.,    8.,    8.,  ...,  204.,  220.,  220.]]])

In [33]:
exprs = b**2 - 4 * a * c

exprs.evaluate(data), exprs.evaluate(data_1), exprs.evaluate(data_2)

(tensor([[-4.0000e+00, -7.0000e+00, -8.0000e+00,  ..., -9.9950e+03,
          -9.9950e+03, -9.9950e+03]]),
 tensor([[-4.0000e+00, -7.0000e+00, -8.0000e+00,  ..., -9.9950e+03,
          -9.9950e+03, -9.9950e+03]]),
 tensor([[-4.0000e+00, -7.0000e+00, -8.0000e+00,  ..., -9.9950e+03,
          -9.9950e+03, -9.9950e+03]]))

In [34]:
cache = {}

def _metric(x, y, w):
    
    global data, data_1, data_2, data_3, data_4
    
    key = y[0]

    if key in cache:
        return cache[key]
    
    token_len = key.count('(') + key.count(')')
    
    expr = eval(key)
        
    if token_len > 100:
        return -1.
                
    factor = expr.evaluate(data)
    
    factor_1 = expr.evaluate(data_1)
    
    factor_2 = expr.evaluate(data_2)
    
    factor_3 = expr.evaluate(data_3)

    factor_4 = expr.evaluate(data_4)

    ic = torch.sum(factor_4 * factor == torch.pow(factor_3, 2)) / data.n_polys
    
    if ic.item() < 1: return 0
                        
    ic = ic * (0.5 * torch.sum((factor == factor_1) & (factor != 0)) / data.n_polys \
    + 0.5 * torch.sum((factor == factor_2) & (factor != 0)) / data.n_polys)
    
    #print(ic.item())
    
    #print(expr)
    
    #print("&&&&&&&&&&&&&")
        
    
    if ic.item() == 1:
        
        print(expr)
        
    
    cache[key] = ic.item()
    
    return ic


Metric = make_fitness(function=_metric, greater_is_better=True)

In [35]:
from invargen.utils.correlation import batch_pearsonr, batch_spearmanr
from typing import Tuple, Optional

capacity = 20

exprs = [None for _ in range(capacity + 1)]
values = [None for _ in range(capacity + 1)]
single_ics = np.zeros(capacity+1)
mutual_ics = np.identity(capacity+1)
weights = np.zeros(capacity + 1)

size = 0


cache = {}



def _calc_ics(factor, ic_mut_threshold):
    
    global size, values
        
    mutual_ics = []
    
    for i in range(size):
        
        mutual_ic = batch_pearsonr(factor, values[i]).mean().item()
        
        if mutual_ic > ic_mut_threshold:
            
            return None
        
        mutual_ics.append(mutual_ic)
        
    return mutual_ics


def _add_factor(expr, factor, ic_ret, ic_mut):
    
    global size, exprs, values, single_ics, mutual_ics, weights
    
    n = size
    
    exprs[n] = expr
    
    values[n] = factor
    
    single_ics[n] = ic_ret
    
    for i in range(n):
        
        mutual_ics[i][n] = ic_mut[i]
        
    size += 1
    
    weights[n] = ic_ret
    

def _optimize():
    
    global size, weights
    
    return weights[:size]


def _pop():
    
    global size, capacity, weights
    
    if size <= capacity: return
    
    idx = np.argmin(np.abs(weights))
    
    _swap_idx(idx, capacity)
    
    size = capacity
    

def _swap_idx(i, j):
    
    global size, exprs, values, single_ics, mutual_ics, weights
    
    exprs[i], exprs[j] = exprs[j], exprs[i]
    
    values[i], values[j] = values[j], values[i]
    
    single_ics[i], single_ics[j] = single_ics[j], single_ics[i]
    
    mutual_ics[:, [i, j]] = mutual_ics[:, [j, i]] 
    
    mutual_ics[[i, j], :] = mutual_ics[[j, i], :] 
    
    weights[i], weights[j] = weights[j], weights[i]
    
    
    
    
        

def _metric2(x, y, w):
    
    global data, data_1, data_2, data_3, data_4, exprs, weights
    
    key = y[0]

    if key in cache:
        return cache[key]
    
    token_len = key.count('(') + key.count(')')
    
    expr = eval(key)
        
    if token_len > 100:
        
        return 0.
    
            
    factor = expr.evaluate(data)
    
    factor_1 = expr.evaluate(data_1)
    
    factor_2 = expr.evaluate(data_2)
    
    factor_3 = expr.evaluate(data_3)

    factor_4 = expr.evaluate(data_4)

    ic = torch.sum(factor_4 * factor == torch.pow(factor_3, 2)) / data.n_polys
    
    if ic.item() < 1: return 0
    
    
    ic_ret = ic.item() * (0.5 * torch.sum((factor == factor_1) & (factor != 0)) / data.n_polys \
    + 0.5 * torch.sum((factor == factor_2) & (factor != 0)) / data.n_polys).item()
    
    ic_ret = ic_ret * min(batch_pearsonr(factor, factor_1).mean().item(), batch_pearsonr(factor, factor_2).mean().item())
    
    ic_mut = _calc_ics(factor, ic_mut_threshold=0.99) # if factor is highly correlated to any of the factors in memory return 0
    
    if ic_ret is None or ic_mut is None:
        
        #print(expr, ic_ret)
        
        return 0
    
    
    _add_factor(expr, factor, ic_ret, ic_mut)
    
    if size > 1:
        
        new_weights = _optimize()
        
        worst_idx = np.argmin(np.abs(new_weights))
        
        if worst_idx != capacity:
            
            weights[:size] = new_weights
            
            print(f"[Pool +] {expr}")
            
            if size > capacity:
                
                print(f"[Pool -] {exprs[worst_idx]}")
                
                
        _pop()
        
    #print(ic)
    
    #print(expr)
    
    #print("&&&&&&&&&&&&&")
            
    
    #if ic.item() == 1:
        
        #print(expr)
        
    
    cache[key] = ic_ret
    
    return ic_ret


Metric = make_fitness(function=_metric2, greater_is_better=True)

In [36]:
generation = 0

def ev():
    global generation
    generation += 1
    
    dir_ = 'results'
    os.makedirs(dir_, exist_ok=True)
    if generation % 1 == 0:
        with open(f'{dir_}/{generation}.json', 'w') as f:
            json.dump({'cache': cache}, f)
    
    pass

In [37]:
features = ['a', 'b', 'c']
terminals = features

X_train = np.array([terminals])
y_train = np.array([[1]])

In [38]:
cache = {}

est_gp = SymbolicRegressor(population_size=500,
                           generations=20,
                           init_depth=(2, 8),
                           tournament_size=600,
                           stopping_criteria=1.,
                           p_crossover=0.3,
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.01,
                           p_point_mutation=0.1,
                           p_point_replace=0.6,
                           max_samples=0.9,
                           verbose=1,
                           parsimony_coefficient=0.,
                           random_state=seed,
                           function_set=funcs,
                           metric=Metric,
                           const_range=None,
                           n_jobs=1)

est_gp.fit(X_train, y_train, callback=ev)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
[Pool +] Mul(Constant(4),Add(Mul(Constant(4),$b),Sub($b,$a)))
[Pool +] Mul(Constant(4),$c)
[Pool +] Add(Add(Sub($c,$c),Mul($b,$a)),Mul(Sub($c,$a),Add($a,$a)))
[Pool +] Mul(Mul(Mul($b,$c),$c),$b)
[Pool +] Mul(Mul($a,$a),Sub($c,$c))
[Pool +] Sub(Mul($b,$b),Sub($c,$c))
[Pool +] Mul(Constant(4),$a)
[Pool +] Mul(Sub(Sub(Mul(Constant(4),$a),Mul(Constant(4),$a)),$b),$a)
[Pool +] Mul(Constant(4),Add(Add(Sub(Sub($b,$c),Sub($a,$a)),Sub(Sub($c,$c),Sub($b,$b))),$c))
[Pool +] Add(Add($b,$a),Add($a,$a))
[Pool +] Mul(Add(Sub(Sub($b,$a),Add($c,$b)),Mul(Constant(4),Add($b,$b))),Add(Mul(Mul($b,$c),Add($a,$b)),Mul(Sub($a,$a),Mul(Constant(4),$a))))
[Pool +] Mul(Constant(4),Mul(Add(Sub($b,$c),$b),Mul(Add(Sub(Add($c,$a),Mul(Constant(4),$c)),Mul(Constant(4),Add($a,$c)

[Pool +] Mul(Mul(Mul($a,Add($c,$a)),Add($c,$a)),$c)
[Pool -] Mul($a,Add($c,Mul(Constant(4),$b)))
[Pool +] Mul(Sub($b,Add($b,$a)),$a)
[Pool -] Mul(Add(Mul(Constant(4),$b),Sub($b,$a)),$c)
[Pool +] Sub(Add($b,Sub($b,$a)),$c)
[Pool -] Add(Add($b,Add($c,$a)),$b)
  10     6.95         0.297884       11         0.494431         0.494431      2.78m
[Pool +] Mul(Sub(Sub($a,Sub($c,$c)),Sub($c,$b)),$b)
[Pool -] Sub(Add($b,Sub($b,$a)),$c)
[Pool +] Mul(Mul(Sub($c,Add($c,$b)),Mul($b,$a)),$c)
[Pool -] Add(Mul(Constant(4),$a),Sub($c,$b))
[Pool +] Add($c,$a)
[Pool -] Mul(Mul($a,Mul($a,$a)),$a)
[Pool +] Mul(Mul(Mul($a,Mul($b,$b)),Add($c,$a)),$c)
[Pool -] Mul(Mul($a,Add($a,$a)),$a)
[Pool +] Mul(Mul(Mul(Mul($a,Add($c,$a)),Add($c,$a)),Add($c,$a)),$c)
[Pool -] Sub(Mul($b,$b),Mul($a,$c))
[Pool +] Mul(Mul(Mul($a,Mul(Mul($b,$b),Add($a,$c))),Add($c,$a)),$c)
[Pool -] Add(Mul(Constant(4),Mul($a,$a)),Mul(Mul(Constant(4),$a),Sub($a,$a)))
[Pool +] Mul(Mul(Sub($c,Add($c,$a)),Mul($c,$a)),$c)
[Pool -] Mul(Sub($b,Add($b

SymbolicRegressor(const_range=None,
                  function_set=[<gplearn.functions._Function object at 0x0000023789BBFB50>,
                                <gplearn.functions._Function object at 0x0000023789BBFAC0>,
                                <gplearn.functions._Function object at 0x0000023789BBF730>,
                                <gplearn.functions._Function object at 0x00000237FDB5ED60>],
                  generations=50, init_depth=(2, 8), max_samples=0.9,
                  metric=<gplearn.fitness._Fitness object at 0x000002378C994400>,
                  p_crossover=0.3, p_point_mutation=0.1, p_point_replace=0.6,
                  p_subtree_mutation=0.1, parsimony_coefficient=0.0,
                  population_size=500, random_state=4, stopping_criteria=1.0,
                  tournament_size=600, verbose=1)

In [48]:
exprs, weights

([Sub($a,$c),
  Mul(Add(Mul(Constant(4),Mul(Constant(4),Mul($c,$a))),Mul($a,Mul(Constant(4),Mul($b,Mul(Mul(Constant(4),Sub($b,$b)),Add(Sub($b,$c),$a)))))),$b),
  Mul(Add(Mul(Constant(4),$b),Sub($a,Mul(Constant(4),Mul($b,Mul(Mul(Constant(4),Sub($a,$a)),Sub(Sub($c,$a),$b)))))),$c),
  Mul(Add(Mul(Constant(4),Add($a,$a)),Sub($a,Mul(Constant(4),Add(Mul(Constant(4),Add($a,$a)),Sub($a,Mul(Constant(4),Mul($b,Mul(Mul(Constant(4),Sub($a,$a)),Sub(Sub($c,$a),$b))))))))),$c),
  Add($c,$a),
  Mul(Mul(Mul(Constant(4),Add($c,$a)),Sub($a,Mul(Constant(4),Add($b,Sub(Mul(Constant(4),Sub($a,$a)),Sub(Add($a,$c),$b)))))),$c),
  Add(Mul($c,$b),Mul($b,$a)),
  Add(Mul($c,$c),Mul($a,$a)),
  Mul(Sub(Sub(Mul(Constant(4),$a),Mul(Constant(4),$a)),$b),$a),
  Mul(Constant(4),Add(Add(Sub(Sub($b,$c),Sub($a,$a)),Sub(Sub($c,$c),Sub($b,$b))),$c)),
  Mul(Sub($a,$c),Sub($c,$a)),
  Mul(Mul($b,$c),Mul($a,$c)),
  Mul($b,$a),
  Mul(Add(Mul(Constant(4),Add($a,$b)),Sub($c,Mul(Constant(4),Sub($b,Mul(Mul(Constant(4),Sub($a,$a)),Add(

In [21]:
key = est_gp._program.execute(X_train)[0]

expr = eval(key)



In [17]:
eval('Sub(Mul(b,b),Mul(c,a))')

Sub(Mul($b,$b),Mul($c,$a))

In [39]:
expr = a * c

expr

factor = expr.evaluate(data)

factor_1 = expr.evaluate(data_1)
    
factor_2 = expr.evaluate(data_2)

factor_3 = expr.evaluate(data_3)

factor_4 = expr.evaluate(data_4)

print(torch.sum(factor_4 * factor == torch.pow(factor_3, 2)) / data.n_polys, torch.sum((factor == factor_1) & (factor != 0)) / data.n_polys, torch.sum((factor == factor_2) & (factor != 0)) / data.n_polys)

print(((factor - factor_1) ** 1 / data.n_polys).sum().item(), ((factor - factor_2) ** 1 / data.n_polys).sum().item())

print((batch_pearsonr(factor, factor_1).mean().item()), (batch_pearsonr(factor, factor_2).mean().item()))

tensor(1.) tensor(1.) tensor(0.)
0.0 -564.9691162109375
1.0000001192092896 0.7966890931129456


In [53]:
expr = (a-c) * (c - a)

expr

factor = expr.evaluate(data)

factor_1 = expr.evaluate(data_1)
    
factor_2 = expr.evaluate(data_2)

factor_3 = expr.evaluate(data_3)

factor_4 = expr.evaluate(data_4)

print(torch.sum(factor_4 * factor == torch.pow(factor_3, 2)) / data.n_polys, torch.sum((factor == factor_1) & (factor != 0)) / data.n_polys, torch.sum((factor == factor_2) & (factor != 0)) / data.n_polys)

print(((factor - factor_1) ** 1 / data.n_polys).sum().item(), ((factor - factor_2) ** 1 / data.n_polys).sum().item())

print((batch_pearsonr(factor, factor_1).mean().item()), (batch_pearsonr(factor[:, :10], factor_2[:, :10]).mean().item()))

tensor(1.) tensor(0.9947) tensor(0.)
0.0 2908.2421875
1.0 0.8957033157348633
